In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from scipy.spatial import distance
import scipy.stats as stats

%matplotlib inline

In [2]:
TIME_SAMPLE_FREQ = '30s'

In [3]:
df = pd.read_csv(".\\_events\\_generated\\valid_user_3\\base_wifi_0.data", sep = ';', index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'uuid', 'bssid', 'chwidth', 'freq', 'level', 'user'])

In [4]:
df.head()

,timestamp,uuid,bssid,chwidth,freq,level,user
0,2021-03-09 18:51:38.850,6e33bee6-e4c6-46c5-9b91-32586512120e,b0:95:75:91:08:82,1,2422,-83,3
1,2021-03-09 18:51:38.850,6e33bee6-e4c6-46c5-9b91-32586512120e,2c:56:dc:40:e5:3c,0,2412,-82,3
2,2021-03-09 18:51:38.850,6e33bee6-e4c6-46c5-9b91-32586512120e,88:d7:f6:5e:c5:64,0,2472,-79,3
3,2021-03-09 18:51:38.850,6e33bee6-e4c6-46c5-9b91-32586512120e,52:ff:20:43:22:20,2,5260,-59,3
4,2021-03-09 18:51:38.850,6e33bee6-e4c6-46c5-9b91-32586512120e,50:ff:20:23:22:20,2,5260,-58,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 7 columns):
timestamp    181 non-null object
uuid         181 non-null object
bssid        181 non-null object
chwidth      181 non-null int64
freq         181 non-null int64
level        181 non-null int64
user         181 non-null int64
dtypes: int64(4), object(3)
memory usage: 10.0+ KB


In [6]:
df.index = pd.DatetimeIndex(df.timestamp)
df = df.sort_index()

In [7]:
VALID_USER = df.iloc[0]['user']
df['events_count'] = 1

In [8]:
df = df.drop(['timestamp', 'chwidth'], axis = 1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 181 entries, 2021-03-09 18:51:38.850000 to 2021-03-09 19:01:36.410000
Data columns (total 6 columns):
uuid            181 non-null object
bssid           181 non-null object
freq            181 non-null int64
level           181 non-null int64
user            181 non-null int64
events_count    181 non-null int64
dtypes: int64(4), object(2)
memory usage: 14.9+ KB


In [10]:
bssid_map = { bssid.replace(' ', ''): idx for bssid, idx in zip(df.bssid.unique(), range(len(df.bssid.unique()))) }

In [11]:
df.bssid = df.bssid.apply(lambda x: str(x).replace(' ', ''))
df.level = df.level.apply(lambda x: str(x).replace(' ', ''))
df.freq = df.freq.apply(lambda x: str(x).replace(' ', ''))

In [12]:
df['bssid_level'] = df[['bssid', 'level']].agg(','.join, axis=1)
df['count'] = 1

In [13]:
def user_agg(col):
    if (col == VALID_USER).all():
        return 1
    else:
        return 0

In [14]:
def agg_string_join(col):
    col = col.apply(lambda x: str(x))
    return col.str.cat(sep = ',').replace(' ', '')

In [15]:
def agg_bssid_col(col):
    array_len = len(bssid_map)
    array = np.zeros(array_len, dtype = 'float')
    def fill_array(x):
        tmp = x.split(',')
        bssid = tmp[0]
        level = float(tmp[1])
        array[bssid_map[bssid.replace(' ', '')]] = level
        return
        
    col.apply(lambda x: fill_array(x))
    return np.array2string(array, separator = ',').replace(' ', '')[1:-1]

In [16]:
all_func_dicts_quantum = { 'freq': agg_string_join, 'level': agg_string_join, 'bssid_level' : agg_bssid_col, 'count' : 'sum', 
                         'events_count' : 'sum', 'user' : user_agg}

In [17]:
df_quantum = df.groupby(['timestamp', 'uuid'], as_index=True).agg(all_func_dicts_quantum)

In [18]:
df_quantum = df_quantum.reset_index()
df_quantum.index = pd.DatetimeIndex(df_quantum.timestamp)

In [19]:
df_quantum = df_quantum[df_quantum['count'] != 0]

In [20]:
df_conn = pd.read_csv(".\\_events\\_generated\\valid_user_3\\conn_wifi_0.data", sep = ';', index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', '1', 'bssid', '2', '3', '4', '5', 'level', '6'])

df_conn = df_conn.drop(df_conn.columns.difference(['bssid','timestamp', 'level']), axis = 1)
df_conn.index = pd.DatetimeIndex(df_conn.timestamp)
df_conn = df_conn.sort_index()

In [21]:
def get_level_from_row(row):
    bssid = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['bssid']
    if str(bssid) == 'nan' or str(bssid) == 'null' or str(bssid) == '':
        return 0
    
    level = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['level']
    time = dt.strptime(df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['timestamp'], '%Y-%m-%d %H:%M:%S.%f')
    return level if abs((time - row.name).total_seconds()) <= 10 else 0

# df_quantum['conn_level'] = df_quantum.apply(lambda row: get_level_from_row(row), axis = 1)

In [22]:
def string2array(string):
    try:
        array = np.fromstring(string, sep=',')
        return array
    except:
        return np.nan

def to_ones_array(array):
    try:
        array[array != 0] = 1
        return array
    except:
        return np.nan

def get_len(obj):
    try:
        length = len(obj)
        return length
    except:
        return np.nan

In [23]:
def get_occured_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(curr, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_disappeared_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(prev, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_jaccard_index(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return distance.jaccard(prev, curr)

def get_occur_speed(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))
    
def get_level_speed(row, prev_col, curr_col):
    prev = string2array(row[prev_col])
    curr = string2array(row[curr_col])
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

def calc_single_cols_in_window(df, col, new_col, window, func):
    def func_wrapper(func, row, prev_col, curr_col):
        delta = row.timestamp - row.prev_timestamp
        if pd.isnull(delta):
            delta = 0
        else:
            delta = abs(delta.total_seconds())
        if delta > 10 * 60:
            return np.nan
        else:
            return func(row, prev_col_name, col)
        
    new_cols = []
        
    for i in range(window):
        prev_col_name = "_".join(['prev', col, str(i + 1)])
        new_col_name = "_".join([new_col, str(i + 1)])
        
        df['prev_timestamp'] = df.timestamp.shift(i + 1)
        df[prev_col_name] = df[col].shift(i + 1)
        df[new_col_name] = df.apply(lambda row: func_wrapper(func, row, prev_col_name, col), axis = 1)
        df = df.drop(prev_col_name, axis = 1)
        df = df.drop('prev_timestamp', axis = 1)
        new_cols.append(new_col_name)
        
    df["_".join([new_col, 'mean'])] = df[new_cols].mean(axis = 1)
    df["_".join([new_col, 'median'])] = df[new_cols].median(axis = 1)
    df["_".join([new_col, 'var'])] = df[new_cols].var(axis = 1)
        
    return df

In [24]:
WINDOW_SIZE = 5

occur_and_level_columns_map = [
    ("bssid_level", "occured_nets_count", WINDOW_SIZE, get_occured_nets_count),
    ("bssid_level", "disappeared_nets_count", WINDOW_SIZE, get_disappeared_nets_count),
    ("bssid_level", "jaccard_index", WINDOW_SIZE, get_jaccard_index), 
    ("bssid_level", "occur_speed", WINDOW_SIZE, get_occur_speed),
    ("bssid_level", "level_speed", WINDOW_SIZE, get_level_speed)
]

for (col, new_col, window, func) in occur_and_level_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [25]:
def get_conn_level_speed(row, prev_col, curr_col):
    return row[curr_col] - row[prev_col]

In [26]:
single_columns_map = [
#     ("conn_level", "conn_level_speed", WINDOW_SIZE, get_conn_level_speed),
    ("count", "count_speed", WINDOW_SIZE, get_conn_level_speed)
]

for (col, new_col, window, func) in single_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [27]:
def agg_str(col):
#     all_freq = col.str.cat(sep=',')
    return string2array(col)

def str_mean(col):
    array = agg_str(col)
    if str(array) == 'nan':
        return 0 
    return np.mean(array)

def mean(col):
    return np.mean(col)

def var(col):
    return np.var(col)

def median(col):
    return np.median(col)

def skew(col):
    return stats.skew(col)

def kurt(col):
    return stats.kurtosis(col)

def user_agg(col):
    if (col == 1).all():
        return 1
    else:
        return 0

In [28]:
df_quantum['freq'] = df_quantum.apply(lambda row: str_mean(row['freq']), axis = 1)
df_quantum['level'] = df_quantum.apply(lambda row: str_mean(row['level']), axis = 1)

In [29]:
cols_for_drop = []
names = [
    "occured_nets_count",
    "disappeared_nets_count",
    "jaccard_index",
    "occur_speed",
    "count_speed",
#     "conn_level_speed",
    "level_speed",
    "count_speed"
]

for i in range(1, WINDOW_SIZE + 1):
    for name in names:
        cols_for_drop.append('_'.join([name, str(i)]))
        
df_quantum = df_quantum.drop(['bssid_level', 'timestamp', 'uuid'], axis = 1)
df_quantum = df_quantum.drop(cols_for_drop, axis = 1)

In [30]:
df_quantum.columns

Index(['freq', 'level', 'count', 'events_count', 'user',
       'occured_nets_count_mean', 'occured_nets_count_median',
       'occured_nets_count_var', 'disappeared_nets_count_mean',
       'disappeared_nets_count_median', 'disappeared_nets_count_var',
       'jaccard_index_mean', 'jaccard_index_median', 'jaccard_index_var',
       'occur_speed_mean', 'occur_speed_median', 'occur_speed_var',
       'level_speed_mean', 'level_speed_median', 'level_speed_var',
       'count_speed_mean', 'count_speed_median', 'count_speed_var'],
      dtype='object')

In [31]:
common_cols = [x for x in df_quantum.columns[0:5] if x != 'user' and x != 'events_count']
speed_acc_cols = df_quantum.columns[5:]

common_funcs_list = [mean, var, median, skew, kurt]
special_funcs_list = [mean, pd.DataFrame.mad, skew]

common_cols_map = { col : common_funcs_list for col in common_cols }
speed_acc_cols_map = { col : special_funcs_list for col in speed_acc_cols }

additional = { 'user' : user_agg, 'events_count' : 'sum' }

agg_dict = common_cols_map
agg_dict.update(speed_acc_cols_map)
agg_dict.update(additional)

In [32]:
df_quantum[speed_acc_cols] = df_quantum[speed_acc_cols].apply(pd.to_numeric)

In [33]:
df_sampling = df_quantum.groupby(pd.Grouper(freq = TIME_SAMPLE_FREQ)).agg(agg_dict)

D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
df_rolling = df_quantum.rolling(TIME_SAMPLE_FREQ, min_periods = 1, center = False).agg(agg_dict)

In [38]:
df_sampling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                       for (high_level_name, low_level_name) in df_sampling.columns.values]

df_rolling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                      for (high_level_name, low_level_name) in df_rolling.columns.values]

In [ ]:
df_sampling = df_sampling.dropna()
df_sampling = df_sampling.fillna(0)

df_rolling = df_rolling.dropna()
df_rolling = df_rolling.fillna(0)

In [ ]:
df_sampling.to_csv(".\\_datasets\\5s\\wifi_sampling_dataset_5.csv")
df_rolling.to_csv(".\\_datasets\\5s\\wifi_rolling_dataset_5.csv")